In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

In [2]:
# Hyperparams
N = 342 # num of nodes in hidden layer
lr = 0.001 # learning rate
lambda_ = 1 # Strength of L1 regularization penalty
epochs = 100000

In [3]:
# Define the function we are approximating

def sin_func(x):
    return np.exp(- x ** 2 / 2) * np.sin(7 * np.sqrt(1 + x ** 2))

In [4]:
# Generate random data points
x_train = np.random.uniform(-1, 1, size=(5000,))
y_train = sin_func(x_train)

# Convert the data to PyTorch tensors
x_train = torch.from_numpy(x_train).float()
y_train = torch.from_numpy(y_train).float()

In [5]:
# Define the fully connected neural network with one hidden layer
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(1, N)  # one input feature, 10 hidden units
        self.fc2 = nn.Linear(N, 1)  # 10 hidden units, one output feature
        self.c = nn.Parameter(torch.ones(N))  # Initialize the constant c as a learnable parameter

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.c * x  # Multiply the output of the ReLU activation by c
        x = self.fc2(x)
        return x

In [6]:
# We use L2 Norm on data
criterion = nn.MSELoss()

# We will test both L1 and L2 regularisers
l1_reg = nn.L1Loss()
l2_reg = nn.MSELoss()

# Define our phi function
def phi(input):
    return torch.log(input + 1)

In [7]:
# Define optimizer and instantiate network
net = Net()
optimizer = optim.SGD(net.parameters(), lr=lr)

In [8]:
# Train the network only with L1 reg
for epoch in range(epochs):
    optimizer.zero_grad()
    output = net(x_train.unsqueeze(1))
    loss = criterion(output, y_train.unsqueeze(1))
    l1_loss = l1_reg(net.c, torch.zeros_like(net.c))
    loss += lambda_ * l1_loss  # Add the L1 regularization penalty to the loss function
    loss.backward()
    optimizer.step()
    if epoch % 1000 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item():.4f}")

Epoch 0, Loss: 1.4103


KeyboardInterrupt: 

In [ ]:
# Test the network
x_test = np.linspace(-1, 1, num=100)
y_test = sin_func(x_test)
x_test = torch.from_numpy(x_test).float()
y_test = torch.from_numpy(y_test).float()
output = net(x_test.unsqueeze(1))
loss = criterion(output, y_test.unsqueeze(1))
print(f"Test Loss: {loss.item():.4f}")


In [ ]:
# Plot the results
import matplotlib.pyplot as plt
plt.plot(x_test.numpy(), output.detach().numpy(), label="Predicted")
plt.plot(x_test.numpy(), y_test.numpy(), label="Actual")
plt.legend()
plt.show()

In [ ]:
[p for p in net.parameters()]